# Параллельные вычисления

In [112]:
import csv, ast, time, os
from typing import Dict, Tuple, Iterable, List
from multiprocessing import Process, Queue, cpu_count

try:
    from openpyxl import load_workbook  # Для потокового чтения XLSX
except Exception:
    load_workbook = None  # Если пакет не установлен, остаётся поддержка только CSV



Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`.

In [157]:
from collections import Counter
from typing import Dict
import os

def count_chars_in_file(file_path: str) -> Dict[str, int]:
    """
    Подсчитать, сколько раз встречается каждый символ в файле.
    Регистр игнорируем. Возвращает словарь {символ: количество}.
    """
    char_counts = Counter()
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            char_counts.update(line.lower())
    # можно убрать перевод строки, если не нужен
    if "\n" in char_counts:
        del char_counts["\n"]
    return dict(char_counts)


file1 = "Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt"
file2 = "Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt"
c1 = count_chars_in_file(file1)
c2 = count_chars_in_file(file2)
print(len(c1), len(c2)) #количество символов

19 17


2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс.

In [162]:
import multiprocessing as mp
from typing import Dict, List
from worker_functions import _worker_count

def parallel_count_for_files(file_paths: List[str]) -> Dict[str, Dict[str, int]]:
    """
    Запускает отдельный процесс на каждый файл и собирает результаты
    в multiprocessing.Manager().dict().
    """
    with mp.Manager() as manager: # создаёт менеджер для разделяемых объектов между процессами
        result = manager.dict() # доступный из всех процессов
        procs = []
        for fp in file_paths:
            p = mp.Process(target=_worker_count, args=(fp, result, os.path.basename(fp))) #создаёт новый процесс(функция для выполнения
            p.start()
            procs.append(p)
        for p in procs:
            p.join() #ждём завершения каждого процесса
        return dict(result)


files = [file1, file2]
res = parallel_count_for_files(files)
print(res.keys())

dict_keys(['Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt', 'Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt'])


## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [114]:
from pathlib import Path
DATA_PATH = Path('recipes_full.csv')
OUT_DIR = Path('results/')

N_PARTS = 8

DELIM = ';'

PART_PREFIX = 'id_tag_nsteps_'

OUT_DIR.mkdir(parents=True, exist_ok=True)




In [ ]:
def iter_recipes_unified(path: Path):
    #{'id': int, 'n_steps': int, 'tags': list[str]}
    with open(path, 'r', encoding='utf-8-sig') as f:
        #
        reader = csv.reader(f) 
        header = next(reader)
        
        # Находим индексы колонок
        idx = {name.lower(): i for i, name in enumerate(header)}
        i_id = idx['id']
        i_steps = idx['n_steps']
        i_tags = idx['tags']
        
        for cols in reader:
            rid = int(cols[i_id])
            n_steps = int(cols[i_steps])
            tags = ast.literal_eval(cols[i_tags])
            
            yield {'id': rid, 'n_steps': n_steps, 'tags': tags}

In [ ]:
def split_to_parts_id_tag_nsteps_unified(src_path: Path, out_dir: Path, n_parts: int = 8) -> List[Path]:
    """Читает источники (.xlsx или .csv) потоково и создаёт `n_parts` файлов
    с колонками `id;tag;n_steps`. Заполнение частей идёт по схеме **round-robin** —
    по очереди в каждый файл, чтобы они получались примерно одинакового размера,
    даже если заранее неизвестно общее число строк.
    """
    # Готовим выходные файлы и csv.writer'ы
    out_paths = [out_dir / f"{PART_PREFIX}{i+1}.csv" for i in range(n_parts)]
    writers, files = [], []
    try:
        for p in out_paths: # Открываем все 8 файлов одновременно
            f = open(p, 'w', newline='', encoding='utf-8')
            files.append(f)
            w = csv.writer(f, delimiter=DELIM)
            w.writerow(['id','tag','n_steps'])  # пишем заголовок
            writers.append(w)

        # Индекс текущего файла для round-robin (0..n_parts-1)
        rr = 0

        # Идём по рецептам и раскладываем пары (id, tag, n_steps)
        for rec in iter_recipes_unified(src_path):
            rid = rec['id']
            n_steps = rec['n_steps']
            tags = rec['tags'] or []
            if rid is None or n_steps is None:
                continue  # перестраховка: пропустить странную строку

            for tag in tags:
                if not tag:
                    continue
                t = str(tag).strip()
                if not t:
                    continue
                # Записываем строку и переходим к следующему 
                writers[rr].writerow([rid, t, n_steps])
                rr = (rr + 1) % n_parts #   определяет в какой файл идет запись 
    finally:
        # Гарантированно закрываем все файлы (даже при исключениях)
        for f in files:
            try:
                f.close()
            except Exception:
                pass

    return out_paths


parts = split_to_parts_id_tag_nsteps_unified(DATA_PATH, OUT_DIR, N_PARTS)
print('Созданы части:', [p.name for p in parts])
print('Размеры (байт):', [p.stat().st_size for p in parts])

Созданы части: ['id_tag_nsteps_1.csv', 'id_tag_nsteps_2.csv', 'id_tag_nsteps_3.csv', 'id_tag_nsteps_4.csv', 'id_tag_nsteps_5.csv', 'id_tag_nsteps_6.csv', 'id_tag_nsteps_7.csv', 'id_tag_nsteps_8.csv']
Размеры (байт): [38846621, 38860091, 38844895, 38853250, 38849311, 38854677, 38848220, 38845459]


2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [119]:
from typing import Dict, Tuple

def tag_stats_from_part(csv_path: Path) -> Dict[str, Tuple[int,int]]:
    """Считает промежуточные статистики по одному файлу-части:
       возвращает словарь `tag -> (sum_steps накапливаем, count)`.
       Такой формат удобен для последующего корректного объединения результатов.
    """
    stats: Dict[str, Tuple[int,int]] = {}
    with open(csv_path, 'r', encoding='utf-8', newline='') as f:
        r = csv.reader(f, delimiter=DELIM)
        next(r, None)  # пропускаем заголовок
        for row in r:
            if len(row) != 3:
                continue
            _rid, tag, n_steps = row
            if not tag:
                continue
            try:
                n = int(n_steps)
            except Exception:
                continue
            # Накапливаем сумму шагов и количество вхождений для тега
            s,c = stats.get(tag, (0,0))
            stats[tag] = (s+n, c+1)
    return stats

def tag_means_from_part(csv_path: Path) -> Dict[str, float]:
    """Возвращает `tag -> среднее n_steps` для одного файла."""
    stats = tag_stats_from_part(csv_path)
    return {t: (s / c) for t, (s, c) in stats.items() if c > 0}

# Мини-проверка на первой части
example = tag_means_from_part(parts[0])
print('Пример 10 тегов из', parts[0].name, ':')
for k in list(example.keys())[:10]:
    print(k, f"-> {example[k]:.2f}")

Пример 10 тегов из id_tag_nsteps_1.csv :
mexican -> 5.32
ham-and-bean-soup -> 3.53
quick-breads -> 5.02
60-minutes-or-less -> 9.50
dinner-party -> 8.22
course -> 9.25
chicken -> 7.26
veal -> 3.61
dips-summer -> 3.56
bacon -> 4.00


3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


читаем частичный результат по одному файлу как (суммы, количества) по тегам;
объединяем частичные результаты из многих файлов;
считаем средние;
измеряем время.

In [122]:
def merge_tag_stats(dicts: Iterable[Dict[str, Tuple[int,int]]]) -> Dict[str, Tuple[int,int]]:
    """Складывает несколько `tag -> (sum, count)` в один словарь."""
    merged: Dict[str, Tuple[int,int]] = {}
    for d in dicts:
        for tag, (s, c) in d.items(): # берет результат предидущего файла и к нему прибавляет значения
            S, C = merged.get(tag, (0, 0))
            merged[tag] = (S + s, C + c)
    return merged

def sequential_all_means(files: List[Path]):
    """Последовательно обрабатывает все части и считает финальные средние по тегам."""
    t0 = time.perf_counter()
    parts_stats = [tag_stats_from_part(p) for p in files]
    merged = merge_tag_stats(parts_stats)
    means = {t: (s / c) for t, (s, c) in merged.items() if c > 0}
    dt = time.perf_counter() - t0
    return means, dt

means_seq, t_seq = sequential_all_means(parts)
print(f'Последовательно: {len(parts)} файлов за {t_seq:.4f} с; тегов: {len(means_seq)}')
print('Примеры 10 тегов:')
for k in list(means_seq.keys())[:10]:
    print(k, f"-> {means_seq[k]:.2f}")

Последовательно: 8 файлов за 5.2589 с; тегов: 551
Примеры 10 тегов:
mexican -> 5.30
ham-and-bean-soup -> 3.51
quick-breads -> 5.06
60-minutes-or-less -> 9.41
dinner-party -> 8.23
course -> 9.27
chicken -> 7.33
veal -> 3.68
dips-summer -> 3.48
bacon -> 4.11


4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [177]:
from multiprocessing import Process, Queue, cpu_count
from worker_functions import worker_file_stats
'''
ОСНОВНОЙ ПРОЦЕСС:
1. Создаём Queue()
2. Запускаем 8 процессов - worker_file_stats
РАБОЧИЕ ПРОЦЕССЫ (параллельно):q.put(результат1)...
ОСНОВНОЙ ПРОЦЕСС:
[q.get(), q.get(), q.get()...] ← ждёт 8 результатов
'''


def mp_one_proc_per_file(files: List[Path]):
    """Запускает по **отдельному процессу на каждый файл**.
       Коммуникация через очередь: каждый процесс кладёт в неё словарь статистик.
    """
    t0 = time.perf_counter()
    q = Queue()
    procs = []
    for p in files:
        pr = Process(target=worker_file_stats, args=(str(p), q)) #аргументы: путь к файлу и очередь
        pr.start() #Процесс начинает выполнение НЕЗАВИСИМО от основного процесса
        procs.append(pr)

    # Забираем результаты от всех процессов
    collected = [q.get() for _ in files]

    # Дожидаемся завершения
    for pr in procs:
        pr.join() #гарантирует, что все процессы завершились корректно

    merged = merge_tag_stats(collected)
    means = {t: (s / c) for t, (s, c) in merged.items() if c > 0}
    dt = time.perf_counter() - t0
    return means, dt

means_mp1, t_mp1 = mp_one_proc_per_file(parts)
print(f'mp(one-per-file): {len(parts)} файлов за {t_mp1:.4f} с; тегов: {len(means_mp1)}')

mp(one-per-file): 8 файлов за 21.9336 с; тегов: 551


5. (*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компь ютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы.

In [186]:
from worker_functions import worker_fixed

'''
ОСНОВНОЙ ПРОЦЕСС:
1. Создаём 6 процесса-воркера
2. Кладём 8 файлов в in_q: [f1, f2, f3, f4, f5, f6, f7, f8]
3. Кладём 4 стоп слова: [None, None, None, None]

ПРОЦЕССЫ-ВОРКЕРЫ:
P1: in_q.get() → f1 → обработал → out_q.put(результат1)
    in_q.get() → f7 → обработал → out_q.put(результат5)  
    in_q.get() → None → break (завершился)
и так далее
ОСНОВНОЙ ПРОЦЕСС:
out_q.get() × 8 → [результат1, результат2, ..., результат8]
'''

def mp_fixed_workers(files: List[Path], n_workers: int | None = None):
    """Запускает фиксированное число рабочих процессов (по умолчанию: половина ядер).
       Задания подаются через очередь `in_q`, результаты собираются из `out_q`.
    """
    if n_workers is None:
        n_workers = max(1, cpu_count() // 2) #12/2

    t0 = time.perf_counter()
    in_q, out_q = Queue(), Queue() # пути к файлам, для результатов

    # Стартуем пул рабочих процессов
    procs = [Process(target=worker_fixed, args=(in_q, out_q)) for _ in range(n_workers)] # создаем процессы которые на старте get
    for pr in procs:
        pr.start() # распределяем задания по процессам

    # Подаём задания (пути) в очередь
    for p in files:
        in_q.put(str(p))

    # Отправляем каждому процессу знак None — указание завершиться
    for _ in procs:
        in_q.put(None) 

    # Собираем результаты
    collected = [out_q.get() for _ in files]

    # Дожидаемся завершения всех процессов
    for pr in procs:
        pr.join()

    merged = merge_tag_stats(collected)
    means = {t: (s / c) for t, (s, c) in merged.items() if c > 0}
    dt = time.perf_counter() - t0
    return means, dt

means_mp2, t_mp2 = mp_fixed_workers(parts)
print(f'mp(fixed-workers): {len(parts)} файлов за {t_mp2:.4f} с; тегов: {len(means_mp2)}')
print('Совпадение ключей:', set(means_seq.keys()) == set(means_mp1.keys()) == set(means_mp2.keys()))

mp(fixed-workers): 8 файлов за 39.3828 с; тегов: 551
Совпадение ключей: True


In [175]:
import importlib
import sys

# Удаляем модуль из кеша, чтобы перезагрузить
if 'worker_functions' in sys.modules:
    del sys.modules['worker_functions']
